In [169]:
#API Call to Census 2013 Projected for County Population Density
import matplotlib.pyplot as plt
import pandas as pd
import pandas as dataframe
from matplotlib.lines import Line2D
from pandas.io.json import json_normalize 
#from stats import median
import plotly.plotly as py
from contextlib import suppress
from operator import add
from itertools import chain
import plotly.figure_factory as ff
import plotly.graph_objs as go
import numpy as np
import requests
import time
import json
from collections import Counter
from config import (census_key)
c= census_key
date = "2013"

In [170]:
#State call
url = "https://api.census.gov/data/2013/pep/natstprc?get=STNAME,STATE,POP&DATE=1&for=state:*&for=county*"
stquery_url = f"{url}&key={c}"
stquery_url


'https://api.census.gov/data/2013/pep/natstprc?get=STNAME,STATE,POP&DATE=1&for=state:*&for=county*&key=1b7887ea11cfca56e6b4c789426c501bbadbc235'

In [171]:
st_response = requests.get(stquery_url).json()
st_response

[['STNAME', 'STATE', 'POP', 'DATE', 'state'],
 ['Alabama', '01', '4779736', '1', '01'],
 ['Alaska', '02', '710231', '1', '02'],
 ['Arizona', '04', '6392017', '1', '04'],
 ['Arkansas', '05', '2915918', '1', '05'],
 ['California', '06', '37253956', '1', '06'],
 ['Colorado', '08', '5029196', '1', '08'],
 ['Connecticut', '09', '3574097', '1', '09'],
 ['Delaware', '10', '897934', '1', '10'],
 ['District of Columbia', '11', '601723', '1', '11'],
 ['Florida', '12', '18801310', '1', '12'],
 ['Georgia', '13', '9687653', '1', '13'],
 ['Hawaii', '15', '1360301', '1', '15'],
 ['Idaho', '16', '1567582', '1', '16'],
 ['Illinois', '17', '12830632', '1', '17'],
 ['Indiana', '18', '6483802', '1', '18'],
 ['Iowa', '19', '3046355', '1', '19'],
 ['Kansas', '20', '2853118', '1', '20'],
 ['Kentucky', '21', '4339367', '1', '21'],
 ['Louisiana', '22', '4533372', '1', '22'],
 ['Maine', '23', '1328361', '1', '23'],
 ['Maryland', '24', '5773552', '1', '24'],
 ['Massachusetts', '25', '6547629', '1', '25'],
 ['Mic

In [172]:
#json format to dataframe
df = pd.DataFrame.from_dict(st_response, orient='columns')

In [173]:
#drops the first row from the dataframe
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0)).reset_index(drop=True)
df.columns.name = None

In [174]:
df

,STNAME,STATE,POP,DATE,state
0,Alabama,01,4779736,1,01
1,Alaska,02,710231,1,02
2,Arizona,04,6392017,1,04
3,Arkansas,05,2915918,1,05
4,California,06,37253956,1,06
5,Colorado,08,5029196,1,08
6,Connecticut,09,3574097,1,09
7,Delaware,10,897934,1,10
8,District of Columbia,11,601723,1,11
9,Florida,12,18801310,1,12


In [175]:
#Draw out state id from df
df['STATE'] = df['STATE'].reset_index(drop=True)
st_id = df['STATE'].values
st_id

array(['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48',
       '49', '50', '51', '53', '54', '55', '56', '72'], dtype=object)

In [176]:
#by state by county API call
cnty_url = "https://api.census.gov/data/2013/pep/cty?get=DENSITY,STNAME,CTYNAME,POP&DATE=1&for=county:*"
cnty_url

'https://api.census.gov/data/2013/pep/cty?get=DENSITY,STNAME,CTYNAME,POP&DATE=1&for=county:*'

In [177]:
print(f"{cnty_url}&in=state:{st_id[0]}&key={c}")

https://api.census.gov/data/2013/pep/cty?get=DENSITY,STNAME,CTYNAME,POP&DATE=1&for=county:*&in=state:01&key=1b7887ea11cfca56e6b4c789426c501bbadbc235


In [178]:
requests.get(f"{cnty_url}&in=state:{st_id}&key={c}")

<Response [400]>

In [179]:
main_dict = []

for id in st_id:
    req_url = f"{cnty_url}&in=state:{id}&key={c}"   
    res = requests.get(req_url)
#    main_dict.append(res)    
    def del_none(res):

#    Delete keys with the value ``None`` in a dictionary, recursively.

#    This alters the input so you may wish to ``copy`` the dict first

        for key, value in list(res.items()):
            if value is None:
                del res[key]
            elif isinstance(value, dict):
                del_none(value)
        return main_dict.append(res)
    


In [180]:
    
#     try:       
#         main_dict['first_county_name'] = res['CTYNAME']
#     except (KeyError, IndexError):
#         print(res[0]['CTYNAME'])
    
#    ['DENSITY', 'STNAME', 'CTYNAME', 'POP', 'DATE', 'state', 'county']
    
#     res = requests.get(req_url)
#     main_dict.append(requests.get(req_url))   
#     first_county = res[1]
#     first_county_density = first_county[0]
#     first_county_name = first_county[2]
# #    main_dict[first_county_name] = first_county_density
#     print(first_county_name)
#     print(first_county_density)
print(main_dict)

[]


In [181]:
def parseOutput(main_dict):
    try:
        x = main_dict.json()
        return x
    except:
        pass    

In [161]:
parsed_df = list(map(parseOutput,main_dict))
#parsed_df2 = list(map(parseOutput,parsed_df))
def k(parsed_df):  
    v for k, v in parsed_df.items() if v is not None

SyntaxError: invalid syntax (<ipython-input-161-272d735861d2>, line 4)

In [162]:
# def delete_keys_from_dict(parsed_df, keys):
#     for key in keys:
#         with suppress(KeyError):
#             del parsed_df[None]
#     for value in parsed_df.values():
#         if isinstance(value, MutableMapping):
#              delete_keys_from_dict(value, keys)

In [153]:
#with open('data.json') as json_data:
#    data = json.load(json_data)    
for NoneType in parsed_df: 
    del NoneType['imageData'] 

In [182]:
#parsed_df = list(map(parseOutput,main_dict))
#parsed_df = parsed_df.applymap(lambda x: x[0])
parsed_df

[[['DENSITY', 'STNAME', 'CTYNAME', 'POP', 'DATE', 'state', 'county'],
  ['91.802847054', 'Alabama', 'Autauga County', '54571', '1', '01', '001'],
  ['114.64465484', 'Alabama', 'Baldwin County', '182265', '1', '01', '003'],
  ['31.02916937', 'Alabama', 'Barbour County', '27457', '1', '01', '005'],
  ['36.806354154', 'Alabama', 'Bibb County', '22915', '1', '01', '007'],
  ['88.897949076', 'Alabama', 'Blount County', '57322', '1', '01', '009'],
  ['17.523933503', 'Alabama', 'Bullock County', '10914', '1', '01', '011'],
  ['26.964771288', 'Alabama', 'Butler County', '20947', '1', '01', '013'],
  ['195.69914593', 'Alabama', 'Calhoun County', '118572', '1', '01', '015'],
  ['57.356573651', 'Alabama', 'Chambers County', '34215', '1', '01', '017'],
  ['46.935232365', 'Alabama', 'Cherokee County', '25989', '1', '01', '019'],
  ['62.990178345', 'Alabama', 'Chilton County', '43643', '1', '01', '021'],
  ['15.171314197', 'Alabama', 'Choctaw County', '13859', '1', '01', '023'],
  ['20.858868171', '

In [183]:
df2 = pd.DataFrame(parsed_df)
#df2 = pd.DataFrame.from_dict(parsed_df, orient='columns')
#pd.concat(map(pd.parsed_df,json))

TypeError: object of type 'NoneType' has no len()

In [53]:
#json format to dataframe
#df2 = pd.DataFrame.from_dict(res, orient='columns')

df2 = pd.DataFrame(parsed_df)
#drops the first row from the dataframe
df2.columns = df2.iloc[0]
df2 = df2.reindex(df2.index.drop(0)).reset_index(drop=True)
df2.columns.name = None
df2.tail

TypeError: object of type 'NoneType' has no len()